In [ ]:
!gdown --id 1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF -O disease.zip
import zipfile
import os

zip_file = "disease.zip"  # Name of the downloaded file

# Extract the ZIP file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("/content/extracted_file")  # Change path if needed

print("Extraction completed!")


/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF
From (redirected): https://drive.google.com/uc?id=1-cs3am9gDETODriP5-Ka_RxaXBpA7SLF&confirm=t&uuid=12c02203-d32d-4a6e-851b-6e7736dfa5a6
To: /content/disease.zip
100% 2.49G/2.49G [00:32<00:00, 76.3MB/s]
Extraction completed!


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import mixed_precision

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Enable mixed precision for performance boost
mixed_precision.set_global_policy("mixed_float16")

# Dataset path
dataset_path = "/content/extracted_file/Retinal Fundus Images/train"
img_size = (224, 224)
batch_size = 8
num_classes = 11

# Load training and validation datasets efficiently
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.1,
    subset='training',
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.1,
    subset='validation',
    seed=42
)

# Preprocess using EfficientNet preprocessing
def preprocess(img, label):
    img = tf.cast(img, tf.float32)
    return preprocess_input(img), label

train_ds = train_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess).prefetch(tf.data.AUTOTUNE)

# Build EfficientNetB0 model
def build_model():
    base = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    base.trainable = False  # Freeze base model for transfer learning

    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(num_classes, activation='softmax', dtype='float32')(x)  # Important for mixed precision

    model = Model(inputs=base.input, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize and train model
model = build_model()

callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=callbacks)

# Evaluate model
test_loss, test_accuracy = model.evaluate(val_ds)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Found 20077 files belonging to 11 classes.
Using 18070 files for training.
Found 20077 files belonging to 11 classes.
Using 2007 files for validation.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5857 - loss: 1.0978

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 139s 49ms/step - accuracy: 0.5858 - loss: 1.0977 - val_accuracy: 0.7270 - val_loss: 0.6573
Epoch 2/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6900 - loss: 0.7577

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 73s 32ms/step - accuracy: 0.6900 - loss: 0.7577 - val_accuracy: 0.7359 - val_loss: 0.6074
Epoch 3/20
2258/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7146 - loss: 0.6856

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 72s 32ms/step - accuracy: 0.7146 - loss: 0.6856 - val_accuracy: 0.7583 - val_loss: 0.5739
Epoch 4/20
2257/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7197 - loss: 0.6514

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 74s 33ms/step - accuracy: 0.7197 - loss: 0.6514 - val_accuracy: 0.7623 - val_loss: 0.5570
Epoch 5/20
2257/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7286 - loss: 0.6357

...

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 80s 31ms/step - accuracy: 0.7476 - loss: 0.5898 - val_accuracy: 0.7663 - val_loss: 0.5129
Epoch 8/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7439 - loss: 0.5874

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 68s 30ms/step - accuracy: 0.7439 - loss: 0.5874 - val_accuracy: 0.7887 - val_loss: 0.4918
Epoch 9/20
2258/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7475 - loss: 0.5763

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 72s 32ms/step - accuracy: 0.7475 - loss: 0.5763 - val_accuracy: 0.7902 - val_loss: 0.4706
Epoch 10/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 77s 30ms/step - accuracy: 0.7534 - loss: 0.5696 - val_accuracy: 0.7818 - val_loss: 0.4884
Epoch 11/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 84s 31ms/step - accuracy: 0.7640 - loss: 0.5488 - val_accuracy: 0.7808 - val_loss: 0.4960
Epoch 12/20
2257/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7623 - loss: 0.5446

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 85s 32ms/step - accuracy: 0.7623 - loss: 0.5446 - val_accuracy: 0.7982 - val_loss: 0.4615
Epoch 13/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 77s 30ms/step - accuracy: 0.7595 - loss: 0.5459 - val_accuracy: 0.7927 - val_loss: 0.4696
Epoch 14/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 73s 32ms/step - accuracy: 0.7694 - loss: 0.5357 - val_accuracy: 0.7952 - val_loss: 0.4748
Epoch 15/20
2257/2259 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7675 - loss: 0.5370

2259/2259 ━━━━━━━━━━━━━━━━━━━━ 72s 32ms/step - accuracy: 0.7675 - loss: 0.5370 - val_accuracy: 0.7927 - val_loss: 0.4604
Epoch 16/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 81s 32ms/step - accuracy: 0.7679 - loss: 0.5293 - val_accuracy: 0.7862 - val_loss: 0.4765
Epoch 17/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 82s 32ms/step - accuracy: 0.7721 - loss: 0.5220 - val_accuracy: 0.7962 - val_loss: 0.4742
Epoch 18/20
2259/2259 ━━━━━━━━━━━━━━━━━━━━ 78s 30ms/step - accuracy: 0.7715 - loss: 0.5151 - val_accuracy: 0.7942 - val_loss: 0.4757
251/251 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.7999 - loss: 0.4512
Test Accuracy: 79.27%
